In [1]:
import numpy as np
import pandas as pd

# Configuraciones iniciales
np.random.seed(42)  # Para reproducibilidad
g = 9.81  # Gravedad en m/s^2
num_samples = 5000  # Número de trayectorias

# Generación de valores aleatorios para las variables
velocidades_iniciales = np.random.uniform(10, 100, num_samples)  # m/s
angulos = np.random.uniform(10, 80, num_samples)  # grados
masas = np.random.uniform(0.5, 10, num_samples)  # kg
constantes_resistencia = np.random.uniform(0.1, 1.0, num_samples)  # kg/m


# Método numérico para simular trayectorias (Euler)
def simular_trayectoria(v0, theta, m, c, dt=0.01, max_time=10):
    theta_rad = np.radians(theta)  # Convertir ángulo a radianes
    vx = v0 * np.cos(theta_rad)  # Componente horizontal de la velocidad
    vy = v0 * np.sin(theta_rad)  # Componente vertical de la velocidad

    # Posiciones iniciales
    x, y = 0, 0
    trayectoria = []

    for _ in np.arange(0, max_time, dt):
        ax = -(c / m) * vx  # Aceleración en x
        ay = -g - (c / m) * vy  # Aceleración en y

        # Actualizar velocidades
        vx += ax * dt
        vy += ay * dt

        # Actualizar posiciones
        x += vx * dt
        y += vy * dt

        # Guardar la posición
        trayectoria.append((x, y))

        # Detener si toca el suelo
        if y < 0:
            break

    return trayectoria


# Generación del dataset
data = []
for v0, theta, m, c in zip(
    velocidades_iniciales, angulos, masas, constantes_resistencia
):
    trayectoria = simular_trayectoria(v0, theta, m, c)
    for punto in trayectoria:
        data.append([v0, theta, m, c, punto[0], punto[1]])  # Guardar datos

# Convertir a DataFrame
columnas = [
    "velocidad_inicial",
    "angulo",
    "masa",
    "constante_resistencia",
    "pos_x",
    "pos_y",
]
datos = pd.DataFrame(data, columns=columnas)

# Guardar el dataset en un archivo CSV
datos.to_csv("trayectorias_proyectil.csv", index=False)

print("Generación de datos completada. Guardado en 'trayectorias_proyectil.csv'.")


Generación de datos completada. Guardado en 'trayectorias_proyectil.csv'.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Cargar el dataset generado
datos = pd.read_csv("trayectorias_proyectil.csv")

# Separar características (X) y etiquetas (y)
X = datos[["velocidad_inicial", "angulo", "masa", "constante_resistencia"]].values
y = datos[["pos_x", "pos_y"]].values

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Crear el modelo de red neuronal profunda
model = Sequential()

# Capa de entrada y primera capa oculta
model.add(Dense(128, input_dim=4, activation="relu"))
# Capas ocultas adicionales
model.add(Dense(128, activation="relu"))
model.add(Dense(128, activation="relu"))

# Capa de salida
model.add(Dense(2, activation="linear"))

# Compilar el modelo
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

# Entrenar el modelo
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1,
)

# Evaluar el modelo
loss, mae = model.evaluate(X_test, y_test)
print(f"Pérdida (Loss): {loss}, Error Absoluto Medio (MAE): {mae}")

# Predicciones en el conjunto de prueba
predicciones = model.predict(X_test)

# Visualizar las predicciones vs valores reales (pos_x y pos_y separadamente)
plt.figure(figsize=(12, 6))

# Comparar pos_x
plt.subplot(1, 2, 1)
plt.scatter(y_test[:, 0], predicciones[:, 0], alpha=0.5)
plt.title("Comparación de pos_x (Real vs Predicho)")
plt.xlabel("Real")
plt.ylabel("Predicho")
plt.grid(True)

# Comparar pos_y
plt.subplot(1, 2, 2)
plt.scatter(y_test[:, 1], predicciones[:, 1], alpha=0.5)
plt.title("Comparación de pos_y (Real vs Predicho)")
plt.xlabel("Real")
plt.ylabel("Predicho")
plt.grid(True)

plt.tight_layout()
plt.show()

# Curvas de aprendizaje
plt.figure(figsize=(10, 6))
plt.plot(history.history["loss"], label="Pérdida de Entrenamiento")
plt.plot(history.history["val_loss"], label="Pérdida de Validación")
plt.title("Curvas de Pérdida durante el Entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Pérdida (Loss)")
plt.legend()
plt.grid(True)
plt.show()